In [1]:
from machine_learning.xgb_def import XGBOptimizer
from machine_learning.reglog_def import REGLOGOptimizer
import pandas as pd
from opt_funct.data_fuct import functions
from sklearn.model_selection import train_test_split
import time
import optuna
from sklearn.metrics import f1_score, classification_report
from machine_learning.combination1 import Combined_xgb_logreg
from machine_learning.SVM_defopt import SVMOptimizer

In [2]:
df = functions.clean_ds(pd.read_csv('./data/aapl_5m_train.csv'))
df

,Unnamed: 0,Timestamp,Gmtoffset,Datetime,Open,High,Low,Close,Volume,X_t-1,X_t-2,X_t-3,Pt_5,RSI,Y_BUY,Y_SELL
0,0,1609770600,0,2021-01-04 14:30:00,133.570007,133.611602,132.389999,132.809997,6624663.0,NaN,NaN,NaN,131.448898,NaN,False,True
1,1,1609770900,0,2021-01-04 14:35:00,132.750000,132.750000,131.809997,131.889999,2541553.0,132.809997,NaN,NaN,131.119995,NaN,False,True
2,2,1609771200,0,2021-01-04 14:40:00,131.500000,132.339996,131.500000,132.059997,2492415.0,131.889999,132.809997,NaN,130.875000,NaN,False,True
3,3,1609771500,0,2021-01-04 14:45:00,132.000000,132.250000,131.899993,132.250000,1859131.0,132.059997,131.889999,132.809997,130.509902,NaN,False,True
4,4,1609771800,0,2021-01-04 14:50:00,132.000000,132.018096,131.520004,131.589996,1780105.0,132.250000,132.059997,131.889999,130.889999,NaN,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39574,19788,1672432800,0,2022-12-30 20:40:00,129.029998,129.130004,128.919998,129.054992,833767.0,129.039993,129.020004,128.910003,NaN,56.509324,False,False
39575,19789,1672433100,0,2022-12-30 20:45:00,129.059997,129.399993,129.020004,129.380004,2136910.0,129.054992,129.039993,129.020004,NaN,60.700323,False,False
39576,19790,1672433400,0,2022-12-30 20:50:00,129.375000,129.929992,129.330001,129.764404,2879267.0,129.380004,129.054992,129.039993,NaN,64.854367,False,False
39577,19791,1672433700,0,2022-12-30 20:55:00,129.764999,129.949996,129.619995,129.949996,3896812.0,129.764404,129.380004,129.054992,NaN,66.620919,False,False


**Entrenamos nuestros modelos para compra**

In [3]:
close_data = df[['Close', 'X_t-1', 'X_t-2', 'X_t-3', 'RSI', 'Y_BUY']].dropna()
x = close_data.drop(columns=['Y_BUY'])
y = close_data[['Y_BUY']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, shuffle=False)

In [4]:
xgboost_optimizer = XGBOptimizer(x_train, y_train, x_test, y_test)
xgboost_best_params, best_model_xgboost = xgboost_optimizer.xgb_optuna()

reg_log_optimizer = REGLOGOptimizer(x_train, y_train, x_test, y_test)
reg_log_best_params, best_model_reg_log = reg_log_optimizer.reglog_optuna()

SVC_optimizer = SVMOptimizer(x_train, y_train, x_test, y_test)
SVC_best_params, best_model_SVC = SVC_optimizer.svm_optuna()
# xgboost_opt = XGBOptimizer(x_train, y_train, x_test, y_test).xgb_optuna()
# reg_log_opt = REGLOGOptimizer(x_train, y_train, x_test, y_test).reglog_optuna()
# combined_opt = Combined_xgb_logreg(x_train, y_train, x_test, y_test).combined_optuna()

[I 2024-03-08 19:39:46,789] A new study created in memory with name: no-name-021fecde-574f-4295-a4e2-7465f67dedf9
/Users/rixon/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [19:39:46] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "gamma", "max_depth", "max_leaves" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-03-08 19:39:46,853] Trial 0 finished with value: 0.678879670360265 and parameters: {'n_estimators': 84, 'max_depth': 29, 'max_leaves': 86, 'learning_rate': 0.5330190811970162, 'booster': 'gblinear', 'gamma': 38.2996321227501, 'reg_alpha': 1.136531506032837, 'reg_lambda': 0.13084739160255524}. Best is trial 0 with value: 0.678879670360265.
/Users/rixon/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [19:39:46] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:742: 
Parameters: { "gamma", "max_depth", "max_leaves" } are not used.

  warnings.warn(smsg, UserWarning)
[I 2024-03-

F1: 0.678879670360265
Best hyperparameters: {'n_estimators': 84, 'max_depth': 29, 'max_leaves': 86, 'learning_rate': 0.5330190811970162, 'booster': 'gblinear', 'gamma': 38.2996321227501, 'reg_alpha': 1.136531506032837, 'reg_lambda': 0.13084739160255524}
Execution time: 1.011798580487569 minutes


[I 2024-03-08 19:40:49,699] Trial 0 finished with value: 0.581754590498397 and parameters: {'C': 7.240109058449951, 'fit_intercept': False, 'l1_ratio': 0.5842244266346386}. Best is trial 0 with value: 0.581754590498397.
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2024-03-08 19:40:51,873] Trial 1 finished with value: 0.5815664845173042 and parameters: {'C': 4.023749824226845, 'fit_intercept': False, 'l1_ratio': 0.9822783407325992}. Best is trial 0 with value: 0.581754590498397.
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2024-0

F1 Score: 0.6008631491020465
Best hyperparameters: {'C': 0.003396628736677583, 'fit_intercept': True, 'l1_ratio': 0.7916820522517427}
Execution time: 2.9798348824183147 minutes


[I 2024-03-08 19:43:46,733] A new study created in memory with name: no-name-a9ada387-bd76-4cf8-a22c-9b803aa2fd0c
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2024-03-08 19:43:49,375] Trial 0 finished with value: 0.4737618180241131 and parameters: {'C': 3.258750578665133, 'kernel': 'linear', 'degree': 2, 'gamma': 5.0}. Best is trial 0 with value: 0.4737618180241131.
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. P

F1 Score: 0.678879670360265
Best hyperparameters: {'C': 6.7308410512150205, 'kernel': 'sigmoid', 'degree': 2, 'gamma': 'auto'}
Execution time: 16.956160219510398 minutes


In [5]:

close_data['xg_boost_buy'] = best_model_xgboost.predict(x)
close_data['reg_log_buy'] = best_model_reg_log.predict(x)
close_data['SVC_buy'] = best_model_SVC.predict(x)
close_data

,Close,X_t-1,X_t-2,X_t-3,RSI,Y_BUY,xg_boost_buy,reg_log_buy,SVC_buy
27,129.413299,129.390106,129.690002,129.910003,28.060866,False,1,False,True
28,129.148498,129.413299,129.390106,129.690002,26.485360,False,1,False,True
29,128.820007,129.148498,129.413299,129.390106,24.701195,False,1,False,True
30,128.580001,128.820007,129.148498,129.413299,23.501640,False,1,False,True
31,128.351699,128.580001,128.820007,129.148498,22.427254,False,1,False,True
...,...,...,...,...,...,...,...,...,...
39574,129.054992,129.039993,129.020004,128.910003,56.509324,False,1,True,True
39575,129.380004,129.054992,129.039993,129.020004,60.700323,False,1,True,True
39576,129.764404,129.380004,129.054992,129.039993,64.854367,False,1,True,True
39577,129.949996,129.764404,129.380004,129.054992,66.620919,False,1,True,True


In [7]:
close_data2 = df[['Close', 'X_t-1', 'X_t-2', 'X_t-3', 'RSI', 'Y_SELL']].dropna()
x = close_data2.drop(columns=['Y_SELL'])
y = close_data2[['Y_SELL']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, shuffle=False)

In [8]:
xgboost_optimizer = XGBOptimizer(x_train, y_train, x_test, y_test)
xgboost_best_params, best_model_xgboost = xgboost_optimizer.xgb_optuna()

reg_log_optimizer = REGLOGOptimizer(x_train, y_train, x_test, y_test)
reg_log_best_params, best_model_reg_log = reg_log_optimizer.reglog_optuna()

SVC_optimizer = SVMOptimizer(x_train, y_train, x_test, y_test)
SVC_best_params, best_model_SVC = SVC_optimizer.svm_optuna()

[I 2024-03-08 20:23:28,605] A new study created in memory with name: no-name-f034767a-0a39-429b-a951-126fb49cb04d
[I 2024-03-08 20:23:30,876] Trial 0 finished with value: 0.0 and parameters: {'n_estimators': 84, 'max_depth': 5, 'max_leaves': 28, 'learning_rate': 1.0834420895224774, 'booster': 'dart', 'gamma': 22.245313313862663, 'reg_alpha': 9.30081295102253, 'reg_lambda': 7.077726678913676}. Best is trial 0 with value: 0.0.
[I 2024-03-08 20:23:30,894] Trial 1 finished with value: 0.0 and parameters: {'n_estimators': 7, 'max_depth': 53, 'max_leaves': 33, 'learning_rate': 1.5801546571563627, 'booster': 'gbtree', 'gamma': 47.738672016570206, 'reg_alpha': 7.551442246917899, 'reg_lambda': 1.5655719479806829}. Best is trial 0 with value: 0.0.
[I 2024-03-08 20:23:35,844] Trial 2 finished with value: 0.0 and parameters: {'n_estimators': 127, 'max_depth': 45, 'max_leaves': 65, 'learning_rate': 1.8412553548384893, 'booster': 'dart', 'gamma': 21.02668371701165, 'reg_alpha': 6.088160788310286, 'r

F1: 0.6162561262809496
Best hyperparameters: {'n_estimators': 66, 'max_depth': 77, 'max_leaves': 52, 'learning_rate': 1.723563288176993, 'booster': 'dart', 'gamma': 12.403042250179183, 'reg_alpha': 1.7495501035630716, 'reg_lambda': 8.528885425119606}
Execution time: 1.3028055826822917 minutes


[I 2024-03-08 20:24:48,551] A new study created in memory with name: no-name-e18ab9b6-7092-470a-b791-f853a1233eae
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2024-03-08 20:24:50,403] Trial 0 finished with value: 0.32076600837821667 and parameters: {'C': 0.09056499008554898, 'fit_intercept': False, 'l1_ratio': 0.5373272713476821}. Best is trial 0 with value: 0.32076600837821667.
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
[I 2024-03-08 20:24:52,485] Trial 1 finished with value: 0.32235743259365307 and parameters: {'C': 7.160683166

F1 Score: 0.3398205337373267
Best hyperparameters: {'C': 2.927408239255935, 'fit_intercept': False, 'l1_ratio': 0.07467756756902702}
Execution time: 3.400634598731995 minutes


/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2024-03-08 20:28:21,528] Trial 0 finished with value: 0.0 and parameters: {'C': 6.37337711341939, 'kernel': 'sigmoid', 'degree': 2, 'gamma': 2.0}. Best is trial 0 with value: 0.0.
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/rixon/anaconda3/lib/python3.11/site-packages/sklearn/svm/_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
[I 2024-03-08 20:28:34,519] Trial 1 finished with value: 0.4563552833078101 and para

F1 Score: 0.6511945392491467
Best hyperparameters: {'C': 0.9516750970731981, 'kernel': 'poly', 'degree': 2, 'gamma': 'scale'}
Execution time: 11.629093285401662 minutes


In [9]:
close_data['Y_SELL'] = close_data2['Y_SELL']
close_data['xg_boost_sell'] = best_model_xgboost.predict(x)
close_data['reg_log_sell'] = best_model_reg_log.predict(x)
close_data['SVC_sell'] = best_model_SVC.predict(x)
close_data

,Close,X_t-1,X_t-2,X_t-3,RSI,Y_BUY,xg_boost_buy,reg_log_buy,SVC_buy,Y_SELL,xg_boost_sell,reg_log_sell,SVC_sell
27,129.413299,129.390106,129.690002,129.910003,28.060866,False,1,False,True,True,1,True,False
28,129.148498,129.413299,129.390106,129.690002,26.485360,False,1,False,True,True,1,True,False
29,128.820007,129.148498,129.413299,129.390106,24.701195,False,1,False,True,True,1,True,False
30,128.580001,128.820007,129.148498,129.413299,23.501640,False,1,False,True,True,1,True,False
31,128.351699,128.580001,128.820007,129.148498,22.427254,False,1,False,True,True,1,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39574,129.054992,129.039993,129.020004,128.910003,56.509324,False,1,True,True,False,1,False,False
39575,129.380004,129.054992,129.039993,129.020004,60.700323,False,1,True,True,False,0,False,False
39576,129.764404,129.380004,129.054992,129.039993,64.854367,False,1,True,True,False,1,False,False
39577,129.949996,129.764404,129.380004,129.054992,66.620919,False,1,True,True,False,1,False,False
